Mount Google Drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dataset Link: https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip

Reference Link: https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

Downloading images.tar file

In [23]:
 !wget --no-check-certificate \
     https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip

--2021-06-09 03:28:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/x-httpd-php]
Saving to: ‘UCI HAR Dataset.zip.1’

UCI HAR Dataset.zip 100%[===================>]  58.17M  66.6MB/s    in 0.9s    

2021-06-09 03:28:44 (66.6 MB/s) - ‘UCI HAR Dataset.zip.1’ saved [60999314/60999314]



Extract downloaded UCI HAR Dataset.zip file to required GDrive folder

In [24]:
import shutil

In [25]:
shutil.unpack_archive("UCI HAR Dataset.zip")

In [26]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

print("Packages loaded successfully")

Packages loaded successfully


In [27]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [28]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [29]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix='/content/UCI HAR Dataset/'):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [30]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [31]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [32]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [33]:
# filepath = "/content/UCI HAR Dataset"

In [34]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
run_experiment()



(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 92.195
>#2: 89.990
>#3: 91.619
>#4: 90.193
>#5: 90.804
>#6: 91.415
>#7: 89.956
>#8: 88.090
>#9: 90.058
>#10: 89.175
[92.19545125961304, 89.98982310295105, 91.6185975074768, 90.19341468811035, 90.80420732498169, 91.41499996185303, 89.95589017868042, 88.08958530426025, 90.05768299102783, 89.17543292045593]
Accuracy: 90.350% (+/-1.151)


In [48]:
!pip install pytube
from pytube import YouTube
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [49]:
def download_video(link, path=''):
  yt = YouTube(link)
  print(yt.title)
  yt.streams.all()
  stream = yt.streams.first()
  print("Streaming: ",stream)
  print("Downloading video...")
  download = stream.download()
  filename = download[9:]
  print("Video downloaded succesfully: ",filename)
  if(path != ''):
    shutil.move(filename, path)
    print("File moved to: ", path)


In [50]:
link = "https://www.youtube.com/watch?v=cP-FUOxkc2k" # enter our youtube link here
path = "/content/drive/MyDrive/AmritaUniversity/SecondSemester/CV/ActivityRecognition" # enter the path to move video file

In [53]:
download_video(link, path) 

Waymo Open Dataset available for autonomous vehicle researchers


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  after removing the cwd from sys.path.


Streaming:  <Stream: itag="18" mime_type="video/mp4" res="360p" fps="10fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
Video downloaded succesfully Waymo Open Dataset available for autonomous vehicle researchers.mp4
File moved to:  /content/drive/MyDrive/AmritaUniversity/SecondSemester/CV/ActivityRecognition
